In [1]:
!pip install --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [3]:
import json
from typing import List, Dict, Any, Tuple

from google.colab import auth
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

# Authenticate user in Colab
auth.authenticate_user()

# After auth, Colab stores credentials in a default place.
# We can reuse them to build the Docs API client.
from google.auth.transport.requests import Request
from google.auth import default

creds, _ = default(scopes=["https://www.googleapis.com/auth/documents"])
creds.refresh(Request())

docs_service = build("docs", "v1", credentials=creds)
drive_service = build("drive", "v3", credentials=creds)

print("✅ Authenticated and Docs/Drive clients ready.")


✅ Authenticated and Docs/Drive clients ready.


In [4]:
markdown_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
""".strip()


In [6]:
import re

def parse_markdown(md: str) -> List[Dict[str, Any]]:
    """
    Very lightweight markdown parser tailored for the given meeting notes format.
    Supports:
      - #, ##, ### headings
      - Bullets starting with -, * (with indentation)
      - Checkbox bullets: - [ ] or * [ ]
      - Footer after '---'
    """
    lines = md.splitlines()
    blocks: List[Dict[str, Any]] = []

    in_footer = False
    footer_lines: List[str] = []

    for raw_line in lines:
        line = raw_line.rstrip("\n")

        # Detect footer start (line with ---)
        if re.match(r"^\s*---\s*$", line):
            in_footer = True
            continue

        if in_footer:
            if line.strip():
                footer_lines.append(line)
            continue

        # Headings
        heading_match = re.match(r"^(#{1,3})\s+(.*)$", line)
        if heading_match:
            level = len(heading_match.group(1))
            text = heading_match.group(2).strip()
            blocks.append({
                "type": "heading",
                "level": level,
                "text": text
            })
            continue

        # Bullets (including checkbox)
        bullet_match = re.match(r"^(\s*)[-*]\s+(.*)$", line)
        if bullet_match:
            indent_spaces = len(bullet_match.group(1) or "")
            indent_level = indent_spaces // 2  # rough heuristic

            content = bullet_match.group(2).strip()

            checkbox = False
            # Detect markdown checkbox "- [ ]"
            checkbox_match = re.match(r"^\[ \]\s+(.*)$", content)
            if checkbox_match:
                checkbox = True
                content = checkbox_match.group(1).strip()

            blocks.append({
                "type": "bullet",
                "text": content,
                "indent": indent_level,
                "checkbox": checkbox
            })
            continue

        # Plain text lines (skip empty)
        if line.strip():
            blocks.append({
                "type": "text",
                "text": line.strip()
            })

    if footer_lines:
        blocks.append({
            "type": "footer",
            "text": "\n".join(footer_lines)
        })

    return blocks

parsed_blocks = parse_markdown(markdown_notes)
print("Parsed blocks count:", len(parsed_blocks))
for b in parsed_blocks[:10]:
    print(b)


Parsed blocks count: 45
{'type': 'heading', 'level': 1, 'text': 'Product Team Sync - May 15, 2023'}
{'type': 'heading', 'level': 2, 'text': 'Attendees'}
{'type': 'bullet', 'text': 'Sarah Chen (Product Lead)', 'indent': 0, 'checkbox': False}
{'type': 'bullet', 'text': 'Mike Johnson (Engineering)', 'indent': 0, 'checkbox': False}
{'type': 'bullet', 'text': 'Anna Smith (Design)', 'indent': 0, 'checkbox': False}
{'type': 'bullet', 'text': 'David Park (QA)', 'indent': 0, 'checkbox': False}
{'type': 'heading', 'level': 2, 'text': 'Agenda'}
{'type': 'heading', 'level': 3, 'text': '1. Sprint Review'}
{'type': 'bullet', 'text': 'Completed Features', 'indent': 0, 'checkbox': False}
{'type': 'bullet', 'text': 'User authentication flow', 'indent': 1, 'checkbox': False}


In [7]:
ASSIGNEE_COLOR = {
    "default": {
        "color": {
            "rgbColor": {"red": 0.0, "green": 0.2, "blue": 0.7}
        }
    }
}

def find_assignee_ranges(text: str) -> List[Tuple[int, int]]:
    """
    Finds ranges for @mentions like '@sarah' in the given text.
    Returns list of (start_index, end_index) pairs.
    """
    ranges = []
    for match in re.finditer(r"@\w+", text):
        ranges.append((match.start(), match.end()))
    return ranges


In [24]:
from googleapiclient.errors import HttpError

INDENT_PT = 18  # Google Docs default indentation per list level


def create_document(title: str) -> str:
    doc = docs_service.documents().create(body={"title": title}).execute()
    return doc["documentId"]


def build_requests_from_blocks(blocks: List[Dict[str, Any]], base_index: int = 1) -> List[Dict[str, Any]]:
    text_parts = []
    cursor = 0

    heading_ranges = []
    bullet_items = []   # (start, end, is_checkbox, indent)
    assignee_ranges = []
    footer_ranges = []

    # ---------------------------
    # BUILD TEXT + RAW RANGES
    # ---------------------------
    for block in blocks:
        if block["type"] == "heading":
            start = cursor
            line = block["text"] + "\n"
            text_parts.append(line)
            end = cursor + len(line)

            style = (
                "HEADING_1" if block["level"] == 1
                else "HEADING_2" if block["level"] == 2
                else "HEADING_3"
            )
            heading_ranges.append((start, end, style))
            cursor = end

        elif block["type"] == "bullet":
            start = cursor
            line = block["text"] + "\n"  # no tabs
            text_parts.append(line)
            end = cursor + len(line)

            bullet_items.append((start, end, block["checkbox"], block.get("indent", 0)))

            for s, e in find_assignee_ranges(block["text"]):
                assignee_ranges.append((start + s, start + e))

            cursor = end

        elif block["type"] == "text":
            start = cursor
            line = block["text"] + "\n"
            text_parts.append(line)
            cursor += len(line)

        elif block["type"] == "footer":
            text_parts.append("\n")
            cursor += 1

            start = cursor
            line = block["text"] + "\n"
            text_parts.append(line)
            footer_ranges.append((start, start + len(block["text"])))
            cursor += len(line)

    full_text = "".join(text_parts)

    requests = []
    # ---------------------------
    # 1) INSERT TEXT
    # ---------------------------
    requests.append({
        "insertText": {"location": {"index": base_index}, "text": full_text}
    })

    # ---------------------------
    # 2) APPLY HEADINGS
    # ---------------------------
    for s, e, style in heading_ranges:
        requests.append({
            "updateParagraphStyle": {
                "range": {"startIndex": base_index + s, "endIndex": base_index + e},
                "paragraphStyle": {"namedStyleType": style},
                "fields": "namedStyleType"
            }
        })

    # ---------------------------
    # 3) ASSIGNEE STYLING
    # ---------------------------
    for s, e in assignee_ranges:
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": base_index + s, "endIndex": base_index + e},
                "textStyle": {
                    "bold": True,
                    "foregroundColor": ASSIGNEE_COLOR["default"]
                },
                "fields": "bold,foregroundColor"
            }
        })

    # ---------------------------
    # 4) FOOTER STYLE
    # ---------------------------
    for s, e in footer_ranges:
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": base_index + s, "endIndex": base_index + e},
                "textStyle": {"italic": True, "fontSize": {"magnitude": 10, "unit": "PT"}},
                "fields": "italic,fontSize"
            }
        })

    # ---------------------------
    # 5) APPLY INDENT FIRST
    # ---------------------------
    for s, e, is_cb, indent in bullet_items:
        requests.append({
            "updateParagraphStyle": {
                "range": {"startIndex": base_index + s, "endIndex": base_index + e},
                "paragraphStyle": {
                    "indentStart": {"magnitude": indent * INDENT_PT, "unit": "PT"},
                    "indentFirstLine": {"magnitude": indent * INDENT_PT, "unit": "PT"}
                },
                "fields": "indentStart,indentFirstLine"
            }
        })

    # ---------------------------
    # 6) GROUP & APPLY BULLETS
    # ---------------------------
    grouped = []
    if bullet_items:
        gs, ge, gcb, _ = bullet_items[0]
        prev_end = ge
        prev_cb = gcb

        for s, e, cb, _indent in bullet_items[1:]:
            if cb == prev_cb and s == prev_end:
                ge = e
            else:
                grouped.append((gs, ge, prev_cb))
                gs, ge, prev_cb = s, e, cb
            prev_end = e
        grouped.append((gs, ge, prev_cb))

    for s, e, is_cb in grouped:
        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": base_index + s, "endIndex": base_index + e},
                "bulletPreset": "BULLET_CHECKBOX" if is_cb else "BULLET_DISC_CIRCLE_SQUARE"
            }
        })

    return requests


def write_blocks_to_document(doc_id: str, blocks: List[Dict[str, Any]]):
    reqs = build_requests_from_blocks(blocks)
    docs_service.documents().batchUpdate(
        documentId=doc_id,
        body={"requests": reqs}
    ).execute()

    print("✅ Updated:", f"https://docs.google.com/document/d/{doc_id}/edit")


In [25]:
# 1. Parse markdown
blocks = parse_markdown(markdown_notes)
print("Blocks parsed:", len(blocks))

# 2. Create a Google Doc
doc_title = "Product Team Sync - May 15, 2023 (Auto-Generated)"
doc_id = create_document(doc_title)

# 3. Write formatted content
write_blocks_to_document(doc_id, blocks)

print("All done! Open the link above to view the formatted Google Doc.")


Blocks parsed: 45
✅ Updated: https://docs.google.com/document/d/1bRRF0Y918WH1fLae1pM96mJQB964a0Kfl85f6a_Qrts/edit
All done! Open the link above to view the formatted Google Doc.
